In [1]:
from databrew import DataBrew
from glue import GlueWrapper
import boto3
import json
import time
import sys

sys.path.append("/home/glue_user/project_lf/ETL-TDD")

glue = GlueWrapper('us-east-1')
databrew = DataBrew('us-east-1')
path = 's3://test-lf-wm/etl/raw/claim_db/claim/full/202306041621/'


import stage_claim_into_raw
import stage_policyholder_into_raw
import stage_provider_into_raw

# import glue dependencies
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, DataFrame
from etl.paths.components import Bucket


# initialise spark session
sc = SparkContext()
glueContext = GlueContext(sc)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
log4j:WARN No appenders could be found for logger (org.apache.hadoop.metrics2.lib.MutableMetricsFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
/home/glue_us

## Test Execution

## Process First Run

In [3]:
outputs = []

df, path = stage_claim_into_raw.run(glueContext, Bucket.TEST)
# outputs.append( (df, path) )
# df, path = stage_policyholder_into_raw.run(spark, Bucket.TEST)
# outputs.append( (df, path) )
# df, path = stage_provider_into_raw.run(spark, Bucket.TEST)
# outputs.append( (df, path) )

getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)


In [4]:
df

In [5]:
from awsglue import DynamicFrame

dyf = DynamicFrame.fromDF(df, glueContext, "transformed_dyf")

In [5]:
df.printSchema()

root
|-- claim_id: int
|-- policy_holder_id: int
|-- provider_id: int
|-- date_of_service: string
|-- procedure: string
|-- total_procedure_cost: float
|-- medibank_pays: float
|-- medicare_pays: float
|-- excess: float
|-- out_of_pocket: float



In [16]:
import numpy
# flatten the outputs with numpy
dfs = numpy.array(outputs).flatten()[::2]
paths = numpy.array(outputs).flatten()[1::2]

In [6]:
example_path = "s3://test-lf-wm/etl/raw/claim_db/claim/full/202306042339/"

# Crawl
glue.create_crawler(example_path)

glue.start_crawler(example_path)

############## Wait for crawler to finish #####################

glue.wait_for_crawler(example_path)


############## After crawlers finish. #####################

for path in [example_path]:
     
    try: 
        response = databrew.create_dataset(path)

    except Exception as e:
        print(e)
        
    try: 
        response = databrew.create_profile_job(path)
    except Exception as e:
        print(e)
        
    try: 
        pass
        # response = databrew.start_job_run(path)
    except Exception as e:
        print(e)
    
############## Wait for job to finish #####################

databrew.wait_for_job(example_path)

for path in [path]:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")

############## Use data profile. #####################

# databrew.get_dq_results(path)


Couldn't create crawler. Here's why: AlreadyExistsException: 618572314333:test-raw already exists


Waiting for crawler to finish.......................

Exception: Crawler timed out

In [22]:
dfs[0].show()

+--------+----------------+-----------+---------------+----------+--------------------+-------------+-------------+------+-------------+
|claim_id|policy_holder_id|provider_id|date_of_service| procedure|total_procedure_cost|medibank_pays|medicare_pays|excess|out_of_pocket|
+--------+----------------+-----------+---------------+----------+--------------------+-------------+-------------+------+-------------+
|  154100|          536110|     739426|     2022-07-17|Ultrasound|             2295.05|      1377.03|       459.01|229.51|       229.50|
|  325213|          794772|     849574|     2021-06-15|Ultrasound|             4990.06|      2994.04|       998.01|499.01|       499.00|
|  103560|          375441|     558433|     2023-04-16|       MRI|             2801.21|      1680.73|       560.24|280.12|       280.12|
|  765284|          295528|     558433|     2022-02-20|Ultrasound|             1730.48|      1038.29|       346.10|173.05|       173.04|
|   77324|          953938|     945989|  

In [4]:
path

's3://test-lf-wm/etl/raw/claim_db/claim/full/202306041621/'

In [7]:
from awsglue import DynamicFrame


transformed_dyf = DynamicFrame.fromDF(outputs[0][0], glueContext, "transformed_dyf")